In [1]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env
import polars as pl

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)

Extensions to add: alpha_vantage@1.2.3, biztoc@1.2.3, cboe@1.2.3, derivatives@1.2.4, ecb@1.2.3, econometrics@1.2.3, finra@1.2.3, finviz@1.1.3, government_us@1.2.3, intrinio@1.2.4, nasdaq@1.2.3, openbb_charting@2.1.2, quantitative@1.2.3, seeking_alpha@1.2.3, stockgrid@1.2.3, technical@1.2.3, tmx@1.1.3, tradier@1.1.3, wsj@1.2.3
Extensions to remove: derivatives@1.2.3, intrinio@1.2.3

Building...


In [7]:

from humbldata.toolbox.technical.mandelbrot_channel.model import (
    calc_mandelbrot_channel,
    calc_mandelbrot_channel_historical,
)


data = (
    obb.equity.price.historical(
        ["AAPL"],
        provider="yfinance",
        start_date="2020-01-01",
        end_date="2024-01-01",
        adjustment="splits_and_dividends",
    ).to_polars()
).drop(["dividends", "stock_splits"]).with_columns(pl.lit("AAPL").alias("symbol"))

mandelbrot = calc_mandelbrot_channel_historical(  # noqa: ERA001
        data,
        window="1m",
        rv_adjustment=True,
        rv_method="std",
        rv_grouped_mean=False,
        rs_method="RS",
        live_price=False,
    ).collect()

In [8]:
mandelbrot

date,symbol,bottom_price,close_price,top_price
date,str,f64,f64,f64
2020-02-03,"""AAPL""",75.0304,75.080818,80.8237
2020-02-04,"""AAPL""",74.3672,77.559494,82.3451
2020-02-05,"""AAPL""",74.7959,78.191956,82.4866
2020-02-06,"""AAPL""",75.1319,79.106567,82.6979
2020-02-07,"""AAPL""",75.5179,78.031296,82.7061
…,…,…,…,…
2023-12-22,"""AAPL""",191.0371,193.353287,197.7755
2023-12-26,"""AAPL""",191.4981,192.803986,198.2864
2023-12-27,"""AAPL""",191.81,192.903839,198.6853
